In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import time
import os
import shutil
import pathlib
import csv
import time
import copy
import scipy.stats
import subprocess
from numpy.random import randint
working_dir = pathlib.Path().absolute()
os.chdir(working_dir)

In [2]:
def Gaussian_probability(n, sig):
    mu = 2.0
    gp = scipy.stats.norm(mu, sig)
    delta = 2*mu/n
    ticks = np.arange(n)*delta + delta/2.0
    p = gp.pdf(ticks)*delta
    p = p+(1.0-p.sum())/n
    return p

In [3]:
class box():
    def __init__(self, index, x_array, y_array, z_array, res):
        self.res = res
        self.index = index
        
        self.domain = {}
        self.domain["x"] = x_array
        self.domain["y"] = y_array
        self.domain["z"] = z_array
        
        self.bound = {}
        self.bound["x_p"] = np.max(x_array)
        self.bound["x_n"] = np.min(x_array)
        self.bound["y_p"] = np.max(y_array)
        self.bound["y_n"] = np.min(y_array)
        self.bound["z_p"] = np.max(z_array)
        self.bound["z_n"] = np.min(z_array)
        
        self.xdim = self.bound["x_p"] - self.bound["x_n"]
        self.ydim = self.bound["y_p"] - self.bound["y_n"]
        self.zdim = self.bound["z_p"] - self.bound["z_n"]
        self.area = np.round(self.xdim*self.ydim, 3)
        
        self.attach = {}
        self.attach["n"] = {}
        self.attach["e"] = {}
        self.attach["s"] = {}
        self.attach["w"] = {}
        
        self.openings = {}
                
    def random_split(self, unsel_boxes):
        chosen_coord = ""
        if self.xdim < 4.0 and self.ydim >= 4.0:
            chosen_coord = "y"
        elif self.ydim < 4.0 and self.xdim >= 4.0:
            chosen_coord = "x"
        elif self.xdim >= 4.0 and self.ydim >= 4.0:
            chosen_coord = "x" if self.xdim > self.ydim else "y"
        else: 
            print("Error: space too small to be split!")
            raise ValueError('Parameter should...')
        
        cut_pos = self.domain[chosen_coord][8:-8]
        cut = int(np.round(np.random.choice(cut_pos, p=Gaussian_probability(n=len(cut_pos), sig=0.6))/self.res, 0))
        new_array_1 = self.res*np.arange(int(np.round(self.bound[chosen_coord + "_n"]/self.res, 0)), cut+1)
        new_array_2 = self.res*np.arange(cut, int(np.round(self.bound[chosen_coord + "_p"]/self.res, 0))+1)
        
        other_boxes = unsel_boxes.copy()
        box_1_new_ind = self.index
        box_2_new_ind = len(other_boxes)+1
        
        if chosen_coord == "x":
            box_1 = box(box_1_new_ind, new_array_1, self.domain["y"], self.domain["z"], self.res) # box1 is on the left
            box_2 = box(box_2_new_ind, new_array_2, self.domain["y"], self.domain["z"], self.res) # box2 is on the right
            
            if -1 in list(self.attach["n"].keys()):
                box_1.attach["n"][-1] = box_1.domain["x"]
                box_2.attach["n"][-1] = box_2.domain["x"]
            if -3 in list(self.attach["s"].keys()):
                box_1.attach["s"][-3] = box_1.domain["x"]
                box_2.attach["s"][-3] = box_2.domain["x"]
            
            if len(other_boxes) >= 1:
                for b in other_boxes:
                    if self.index in b.attach["s"].keys():
                        del b.attach["s"][self.index]
                        box1_attach = np.intersect1d(np.round(b.domain["x"],2), np.round(new_array_1,2)) # whether b and box1 touch
                        box2_attach = np.intersect1d(np.round(b.domain["x"],2), np.round(new_array_2,2)) # whether b and box2 touch
                        if len(box1_attach) > 1:
                            box_1.attach["n"][b.index] = box1_attach
                            b.attach["s"][box_1_new_ind] = box1_attach
                        if len(box2_attach) > 1:
                            box_2.attach["n"][b.index] = box2_attach
                            b.attach["s"][box_2_new_ind] = box2_attach
                    
                    if self.index in b.attach["n"].keys():
                        del b.attach["n"][self.index]
                        box1_attach = np.intersect1d(np.round(b.domain["x"],2), np.round(new_array_1,2)) # whether b and box1 touch
                        box2_attach = np.intersect1d(np.round(b.domain["x"],2), np.round(new_array_2,2)) # whether b and box2 touch
                        if len(box1_attach) > 1:
                            box_1.attach["s"][b.index] = box1_attach
                            b.attach["n"][box_1_new_ind] = box1_attach
                        if len(box2_attach) > 1:
                            box_2.attach["s"][b.index] = box2_attach
                            b.attach["n"][box_2_new_ind] = box2_attach
            
            box_1.attach["w"] = self.attach["w"]
            box_2.attach["e"] = self.attach["e"]
            box_1.attach["e"] = {box_2_new_ind: self.domain["y"]}
            box_2.attach["w"] = {box_1_new_ind: self.domain["y"]}
            
            return box_1, box_2, other_boxes
        
        else:
            box_1 = box(box_1_new_ind, self.domain["x"], new_array_1, self.domain["z"], self.res) # box1 is lower
            box_2 = box(box_2_new_ind, self.domain["x"], new_array_2, self.domain["z"], self.res) # box1 is upper
            
            if -2 in list(self.attach["e"].keys()):
                box_1.attach["e"][-2] = box_1.domain["y"]
                box_2.attach["e"][-2] = box_2.domain["y"]
            if -4 in list(self.attach["w"].keys()):
                box_1.attach["w"][-4] = box_1.domain["y"]
                box_2.attach["w"][-4] = box_2.domain["y"]
            
            if len(other_boxes) >= 1:
                for b in other_boxes:
                    if self.index in b.attach["e"].keys():
                        del b.attach["e"][self.index]
                        box1_attach = np.intersect1d(np.round(b.domain["y"],2), np.round(new_array_1,2)) # whether b and box1 touch
                        box2_attach = np.intersect1d(np.round(b.domain["y"],2), np.round(new_array_2,2)) # whether b and box2 touch
                        if len(box1_attach) > 1:
                            box_1.attach["w"][b.index] = box1_attach
                            b.attach["e"][box_1_new_ind] = box1_attach
                        if len(box2_attach) > 1:
                            box_2.attach["w"][b.index] = box2_attach
                            b.attach["e"][box_2_new_ind] = box2_attach
                    
                    if self.index in b.attach["w"].keys():
                        del b.attach["w"][self.index]
                        box1_attach = np.intersect1d(np.round(b.domain["y"],2), np.round(new_array_1,2)) # whether b and box1 touch
                        box2_attach = np.intersect1d(np.round(b.domain["y"],2), np.round(new_array_2,2)) # whether b and box2 touch
                        if len(box1_attach) > 1:
                            box_1.attach["e"][b.index] = box1_attach
                            b.attach["w"][box_1_new_ind] = box1_attach
                        if len(box2_attach) > 1:
                            box_2.attach["e"][b.index] = box2_attach
                            b.attach["w"][box_2_new_ind] = box2_attach
            
            box_1.attach["s"] = self.attach["s"]
            box_2.attach["n"] = self.attach["n"]
            box_1.attach["n"] = {box_2_new_ind: self.domain["x"]}
            box_2.attach["s"] = {box_1_new_ind: self.domain["x"]}
        
            return box_1, box_2, other_boxes

In [4]:
class opening():
    def __init__(self, rm_pair, open_type, x_array, y_array, z_array, res):
        self.res = res
        
        self.open_type = open_type
        self.rm_pair = rm_pair
        
        self.bound = {}
        self.bound["x_p"] = np.max(x_array)
        self.bound["x_n"] = np.min(x_array)
        self.bound["y_p"] = np.max(y_array)
        self.bound["y_n"] = np.min(y_array)
        self.bound["z_p"] = np.max(z_array)
        self.bound["z_n"] = np.min(z_array)
        
        self.domain = {}
        self.domain["x"] = x_array
        self.domain["y"] = y_array
        self.domain["z"] = z_array
        
        self.xdim = self.bound["x_p"] - self.bound["x_n"]
        self.ydim = self.bound["y_p"] - self.bound["y_n"]
        self.zdim = self.bound["z_p"] - self.bound["z_n"]

In [5]:
def create_first_room(x_max, x_min, y_max, y_min, res):
    z_num = int(3.0/res+1)
    x_array = res*np.arange(randint(int(np.rint(x_min/res))+1,int(np.rint(x_max/res))+1,size=1))
    y_array = res*np.arange(randint(int(np.rint(y_min/res))+1,int(np.rint(y_max/res))+1,size=1))
    z_array = res*np.arange(z_num)
    return box(0, np.round(x_array,3), np.round(y_array,3), np.round(z_array,3), res)

In [6]:
def create_next_room(boxes):
    area_list = np.array([b.area for b in boxes])
    prob_area = area_list/sum(area_list)
    boxes_copy = boxes.copy()
    sel_box = np.random.choice(boxes_copy, p=prob_area)
    
    while sel_box.xdim < 4.0 and sel_box.ydim < 4.0:
        boxes_copy.remove(sel_box)
        area_list = np.array([b.area for b in boxes_copy])
        prob_area = area_list/sum(area_list)
        sel_box = np.random.choice(boxes_copy, p=prob_area)
    
    other_boxes = boxes.copy()
    other_boxes.remove(sel_box)
    
    newbox1, newbox2, updated_other_boxes = sel_box.random_split(other_boxes)
    
    return updated_other_boxes + [newbox1] + [newbox2]

In [7]:
def create_random_opening(x_array, y_array, z_array, outdoor, res):
    open_x_array = np.array([])
    open_y_array = np.array([])
    open_z_array = np.array([])
    open_type = ''
    all_opening_types = ['space','door','high_window','window']
    type_prob = [0.1,0.2,0.2,0.5]
    if outdoor:
        all_opening_types = ['high_window','window']
        type_prob = [0.3, 0.7]
    if len(z_array) > 1:
        if len(x_array) > 1: 
            open_y_array = y_array
            avail_height = abs(z_array[-1]-z_array[0])
            avail_width = abs(x_array[-1]-x_array[0])
            height = avail_height
            width = avail_width
            open_type = np.random.choice(all_opening_types, p=type_prob)
            if avail_width <= 1.0:
                width = avail_width
                height = np.random.choice([avail_height, 2.0]) # it's narrow so it will either be a path or a door
                open_type = 'space' if height == avail_height else 'door'
                open_x_array = np.arange(x_array[0], x_array[0]+width+res, res)
                open_z_array = np.arange(z_array[0], z_array[0]+height+res, res)
            else:
                if open_type == 'space':
                    height = avail_height
                    max_width = min([2.0, avail_width])
                    width = np.random.choice(np.arange(1.0, max_width+res, res))
                    posible_start_pos = x_array[:-int(width/res)]
                    start_pos = np.random.choice(posible_start_pos)
                    open_x_array = np.arange(start_pos, start_pos+width+res, res)
                    open_z_array = np.arange(0.0, height+res, res)+z_array[0]
                elif open_type == 'door':
                    height = 2.0
                    width = 1.0
                    posible_start_pos = x_array[:-int(width/res)]
                    start_pos = 0.0
                    if len(posible_start_pos)>2:
                        prob = [0.3]+[0.4/(len(posible_start_pos)-2)]*(len(posible_start_pos)-2)+[0.3]
                        start_pos = np.random.choice(posible_start_pos, p=prob)
                    else:
                        start_pos = np.random.choice(posible_start_pos)
                    open_x_array = np.arange(start_pos, start_pos+width+res, res)
                    open_z_array = np.arange(0.0, height+res, res)+z_array[0]
                elif open_type == 'high_window':
                    height = np.random.choice(np.arange(res*2.0, res*3.0+res, res))
                    posible_w = np.arange(res*4.0, avail_width+res, res)
                    width = np.random.choice(posible_w, p=posible_w/sum(posible_w))
                    posible_start_pos = x_array[:-int(width/res)]
                    start_pos = np.random.choice(posible_start_pos)
                    open_x_array = np.arange(start_pos, start_pos+width+res, res)
                    start_z = avail_height - height
                    open_z_array = np.arange(start_z, start_z+height+res, res)+z_array[0]
                elif open_type == 'window':
                    max_width = min([1.5, avail_width]) # the width of an ordinary window should not be wider than 1.5m
                    width = np.random.choice(np.arange(res*4.0, max_width+res, res))
                    posible_start_pos = x_array[:-int(width/res)]
                    start_pos = np.random.choice(posible_start_pos)
                    open_x_array = np.arange(start_pos, start_pos+width+res, res)
                    
                    # for an ordinary outdoor window, the eye level, 2m, should be maintained 
                    sill_height = np.random.choice(np.arange(res*2.0, 1.0+res, res))
                    height = 2.0-sill_height+np.random.choice(np.arange(0.0, 3.0-2.0+res, res)) # 3m-2m = 1m above eye level
                    open_z_array = np.arange(sill_height, sill_height+height+res, res)+z_array[0]
                else:
                    print("ERROR: Wrong Open Type Information!")
                
        if len(y_array) > 1: 
            open_x_array = x_array
            avail_height = abs(z_array[-1]-z_array[0])
            avail_width = abs(y_array[-1]-y_array[0])
            height = avail_height
            width = avail_width
            open_type = np.random.choice(all_opening_types)
            if avail_width <= 1.0:
                width = avail_width
                height = np.random.choice([avail_height, 2.0]) # it's narrow so it will either be a path or a door
                open_type = 'space' if height == avail_height else 'door'
                open_y_array = np.arange(y_array[0], y_array[0]+width+res, res)
                open_z_array = np.arange(z_array[0], z_array[0]+height+res, res)
            else:
                if open_type == 'space':
                    height = avail_height
                    max_width = min([2.0, avail_width])
                    width = np.random.choice(np.arange(1.0, max_width+res, res))
                    posible_start_pos = y_array[:-int(width/res)]
                    start_pos = np.random.choice(posible_start_pos)
                    open_y_array = np.arange(start_pos, start_pos+width+res, res)
                    open_z_array = np.arange(0.0, height+res, res)+z_array[0]
                elif open_type == 'door':
                    height = 2.0
                    width = 1.0
                    posible_start_pos = y_array[:-int(width/res)]
                    start_pos = 0.0
                    if len(posible_start_pos)>2:
                        prob = [0.3]+[0.4/(len(posible_start_pos)-2)]*(len(posible_start_pos)-2)+[0.3]
                        start_pos = np.random.choice(posible_start_pos, p=prob)
                    else:
                        start_pos = np.random.choice(posible_start_pos)
                    open_y_array = np.arange(start_pos, start_pos+width+res, res)
                    open_z_array = np.arange(0.0, height+res, res)+z_array[0]
                elif open_type == 'high_window':
                    height = np.random.choice(np.arange(res*2.0, res*3.0+res, res))
                    posible_w = np.arange(res*4.0, avail_width+res, res)
                    width = np.random.choice(posible_w, p=posible_w/sum(posible_w))
                    posible_start_pos = y_array[:-int(width/res)]
                    start_pos = np.random.choice(posible_start_pos)
                    open_y_array = np.arange(start_pos, start_pos+width+res, res)
                    start_z = avail_height - height
                    open_z_array = np.arange(start_z, start_z+height+res, res) + z_array[0]
                elif open_type == 'window':
                    max_width = min([1.5, avail_width]) # the width of an ordinary window should not be wider than 1.5m
                    width = np.random.choice(np.arange(res*4.0, max_width+res, res))
                    posible_start_pos = y_array[:-int(width/res)]
                    start_pos = np.random.choice(posible_start_pos)
                    open_y_array = np.arange(start_pos, start_pos+width+res, res)
                    
                    # for an ordinary outdoor window, the eye level, 2m, should be maintained 
                    sill_height = np.random.choice(np.arange(res*2.0, 1.0+res, res))
                    height = 2.0-sill_height+np.random.choice(np.arange(0.0, 3.0-2.0+res, res)) # 3m-2m = 1m above eye level
                    open_z_array = np.arange(sill_height, sill_height+height+res, res) + z_array[0]
                else:
                    print("ERROR: Wrong Open Type Information!")
    else:
        open_z_array = z_array
        avail_x_length = abs(x_array[-1]-x_array[0])
        avail_y_length = abs(y_array[-1]-y_array[0])
        open_type = 'atrium'
        x_length = np.random.choice(np.arange(res, avail_x_length+res, res))
        y_length = np.random.choice(np.arange(res, avail_y_length+res, res))
        start_x = np.random.choice(x_array[:-int(x_length/res)])
        start_y = np.random.choice(y_array[:-int(y_length/res)])
        open_x_array = np.arange(start_x, start_x+x_length+res, res)
        open_y_array = np.arange(start_y, start_y+y_length+res, res)
    
    return open_type, open_x_array, open_y_array, open_z_array

In [8]:
def create_opening_0(rooms, res):
    w_rms = []
    for i_w,rm in enumerate(rooms):
        # print("list of keys: ", list(rm.attach["w"].keys()))
        if -4 in list(rm.attach["w"].keys()):
            w_rms.append(i_w)
    probs = np.array([rooms[j].area for j in w_rms])
    probs = probs**2/sum(probs**2)
    rm_ind1 = np.random.choice(w_rms, p = probs)
    array_x = np.array([rooms[rm_ind1].bound["x_n"]])
    array_y = rooms[rm_ind1].attach["w"][-4]
    array_z = rooms[rm_ind1].domain["z"]
    i_o = -4 < 0
    open_type, open_x_array, open_y_array, open_z_array = create_random_opening(array_x, array_y, array_z, i_o, res)
    rm_pair=(rm_ind1, -4)
    opening_0 = opening(rm_pair, open_type, open_x_array, open_y_array, open_z_array, res)
    rooms[rm_ind1].openings['w'] = opening_0
    return rooms, opening_0

In [9]:
def create_opening_on_box(boxes, curr_index, res):
    new_openings = []
    max_num_openings = 3 if np.random.uniform() < 0.5 else 2
    opening_cnt = len(boxes[curr_index].openings.keys())
    
    nesw = ['n', 'e', 's', 'w']
    sel_nesw = [item for item in nesw if item not in list(boxes[curr_index].openings.keys())]
    np.random.shuffle(sel_nesw)
    # print("sel_nesw: ", sel_nesw)
    for face in sel_nesw:
        if opening_cnt < max_num_openings:
            # print("face: ", face)
            oppo_face = ''
            keys = list(boxes[curr_index].attach[face].keys()).copy() # key is the index of the attached other box in all 5 boxes
            np.random.shuffle(keys)
            # print("keys: ", keys)
            for key in keys:
                array_alpha = boxes[curr_index].attach[face][key]
                if len(array_alpha) > 3: # minimum 0.75m door/path between two rooms (4-1)*0.25=0.75
                    if face == 'n':
                        array_x = array_alpha
                        array_y = np.array([boxes[curr_index].bound["y_p"]])
                        oppo_face = 's'
                    elif face == 's':
                        array_x = array_alpha
                        array_y = np.array([boxes[curr_index].bound["y_n"]])
                        oppo_face = 'n'
                    elif face == 'e':
                        array_y = array_alpha
                        array_x = np.array([boxes[curr_index].bound["x_p"]])
                        oppo_face = 'w'
                    else:
                        array_y = array_alpha
                        array_x = np.array([boxes[curr_index].bound["x_n"]])
                        oppo_face = 'e'
                    array_z = boxes[curr_index].domain["z"]
                    if key >= 0:
                        oppo_opens = list(boxes[key].openings.keys())
                        # print("oppo_face: ", oppo_face)
                        # print("oppo_opens: ", oppo_opens)
                        if oppo_face in oppo_opens or len(oppo_opens) >= 3:
                            # print("check!")
                            continue
                    i_o = key < 0
                    open_type, open_x_array, open_y_array, open_z_array = create_random_opening(array_x, array_y, array_z, i_o, res)
                    rm_pair=(boxes[curr_index].index, key)
                    opening_new = opening(rm_pair, open_type, open_x_array, open_y_array, open_z_array, res)
                    boxes[curr_index].openings[face] = opening_new
                    new_openings.append(opening_new)
                    if key >= 0:
                        boxes[key].openings[oppo_face] = opening_new
                    opening_cnt += 1
                    break
    return boxes, new_openings

In [10]:
def sort_rms_by_attach(rooms):
    dic = {"ind": np.arange(len(rooms))}
    list_in = []
    list_out = []
    for rm in rooms:
        att_ind = list(rm.attach["n"].keys())+list(rm.attach["e"].keys())+list(rm.attach["s"].keys())+list(rm.attach["w"].keys())
        num_in = sum(np.array(att_ind) >= 0)
        num_out = sum(np.array(att_ind) < 0)
        list_in.append(num_in)
        list_out.append(num_out)
    dic["in"] = list_in
    dic["out"] = list_out
    df = pd.DataFrame(dic)
    df = df.sort_values(by = ['in', 'out'], ascending = [False, True])
    return df.ind.tolist()

In [11]:
def find_correct_neighbour(opening, axis, all_rooms):
    true_indice = []
    opx1, opx2, opy1, opy2, opz1, opz2, ind1_, ind2_ = opening
    if axis == "x":
        for j, rm in enumerate(all_rooms):
            rx1, rx2, ry1, ry2, rz1, rz2 = rm
            if opx1 == rx1 or opx1 == rx2:
                if opy1 >= ry1 and opy2 <= ry2:
                    true_indice.append(j)
    else:
        for j, rm in enumerate(all_rooms):
            rx1, rx2, ry1, ry2, rz1, rz2 = rm
            if opy1 == ry1 or opy1 == ry2:
                if opx1 >= rx1 and opx2 <= rx2:
                    true_indice.append(j)
    print("Found correct indice: " + str(int(true_indice[0])) + " and " + str(int(true_indice[1])))
    if not len(true_indice) == 2:
        print("Unexpected condition: detacted " + str(len(true_indice)) + " neighbour rooms for this opening! +++++")
    return true_indice

def correct_opening_neighbour(rooms, openings):    
    new_openings = copy.deepcopy(openings)
    old_inds = []
    new_inds = []
    updated = False
    for i, op in enumerate(openings):
        x1, x2, y1, y2, z1, z2, ind1, ind2 = op
        # wrong index not happening in outdoor openings
        if ind1 < 0 or ind2 < 0:
            continue
        
        if x1 == x2:
            rm1, rm2 = rooms[int(ind1)], rooms[int(ind2)]
            if rm1[0] == rm2[1]:
                if x1 == rm1[0]:
                    continue
                else:
                    print("Found and corrected wrong neighbour index =================")
                    new_ids = find_correct_neighbour(op, "x", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            elif rm1[1] == rm2[0]:
                if x1 == rm1[1]:
                    continue
                else:
                    print("Found and corrected wrong neighbour index =================")
                    new_ids = find_correct_neighbour(op, "x", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            else:
                print("Found and corrected wrong neighbour index =================")
                new_ids = find_correct_neighbour(op, "x", rooms)
                new_openings[i][6] = new_ids[0]
                new_openings[i][7] = new_ids[1]
                old_inds.append([ind1, ind2])
                new_inds.append(new_ids)
                updated = True
                
        if y1 == y2:
            rm1, rm2 = rooms[int(ind1)], rooms[int(ind2)]
            if rm1[2] == rm2[3]:
                if y1 == rm1[2]:
                    continue
                else:
                    print("Found and corrected wrong neighbour index =================")
                    new_ids = find_correct_neighbour(op, "y", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            elif rm1[3] == rm2[2]:
                if y1 == rm1[3]:
                    continue
                else:
                    print("Found and corrected wrong neighbour index =================")
                    new_ids = find_correct_neighbour(op, "y", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            else:
                print("Found and corrected wrong neighbour index =================")
                new_ids = find_correct_neighbour(op, "y", rooms)
                new_openings[i][6] = new_ids[0]
                new_openings[i][7] = new_ids[1]
                old_inds.append([ind1, ind2])
                new_inds.append(new_ids)
                updated = True
    if updated:
        return new_openings
    else:
        return openings

In [12]:
def generate_design(num_rms, resolution, result_folder, num_iteration):
    room_file_temp="room_temp.csv"
    opening_temp="opening_temp.csv"
    
    room1 = create_first_room(x_max=15.0, x_min=10.0, y_max=15.0, y_min=10.0, res=resolution)
    room1.attach["n"] = {-1: room1.domain["x"]}
    room1.attach["e"] = {-2: room1.domain["y"]}
    room1.attach["s"] = {-3: room1.domain["x"]}
    room1.attach["w"] = {-4: room1.domain["y"]}
    rooms_initial = [room1]
    for _ in range(num_rms-1):
        rooms_initial = create_next_room(rooms_initial)
    rooms_initial = sorted(rooms_initial, key=lambda rm: rm.index) #**important!**
    
    room_mx = np.array([])
    open_mx = np.array([])
    single_side_exist = True
    too_few_out_opening = True
    while single_side_exist or too_few_out_opening:
        openings = []
        rooms = copy.deepcopy(rooms_initial)
        rooms, opening_0 = create_opening_0(rooms, resolution)
        openings.append(opening_0)
        indices = np.arange(num_rms)
        np.random.shuffle(indices)

        indices = sort_rms_by_attach(rooms)
        for ind in indices:
            rooms, new_openings = create_opening_on_box(rooms, ind, resolution)
            openings = openings + new_openings
        
        room_mx = np.array([[rm.bound["x_n"],rm.bound["x_p"],rm.bound["y_n"],rm.bound["y_p"],rm.bound["z_n"],rm.bound["z_p"]] for rm in rooms])
        open_mx = np.array([[op.bound["x_n"],op.bound["x_p"],op.bound["y_n"],op.bound["y_p"],op.bound["z_n"],op.bound["z_p"],op.rm_pair[0],op.rm_pair[1]] for op in openings])
        open_mx = correct_opening_neighbour(room_mx, open_mx)
        connected_rid = open_mx[:, 6:].flatten().astype(int)
        
        single_side_exist = False
        too_few_out_opening = False
        for rm_id in range(num_rms):
            if connected_rid.tolist().count(rm_id) < 2:
                single_side_exist = True
        min_out_op = 3
        too_few_out_opening = sum(np.array(connected_rid) < 0) < min_out_op # has less than 3 or 4 outdoor opening
        
        if single_side_exist:
            print("Single-sided Room Exists, Regenerate Openings...")
        elif too_few_out_opening:
            print("Too few out openings, Regenerate Openings...")
    
    with open(room_file_temp,"w",newline='') as csvfile_rm: 
        writer_rm = csv.writer(csvfile_rm)
        for k in range(len(room_mx)):
            writer_rm.writerow(room_mx[k])
        writer_rm.writerow([num_iteration])

    with open(opening_temp,"w",newline='') as csvfile_op: 
        writer_op = csv.writer(csvfile_op)
        for k in range(len(open_mx)):
            writer_op.writerow(open_mx[k])
    
    param_folder = result_folder + "/design_iteration_" + str(num_iteration) + "/"
    if not os.path.exists(param_folder):
        os.makedirs(param_folder)

    rm_file_path = param_folder + "design_" + str(num_iteration) + "_room.csv"
    op_file_path = param_folder + "design_" + str(num_iteration) + "_open.csv"

    shutil.copy(room_file_temp, rm_file_path)
    shutil.copy(opening_temp, op_file_path)

In [13]:
def find_new_idx_list(target_folder, num_new_idx):
    missing_idx = []
    fd_list = os.listdir(target_folder)
    idx_list = np.array([int(f.replace("design_iteration_","")) for f in fd_list])
    idx_list.sort()
    min_idx = np.amin(idx_list)
    max_idx = np.amax(idx_list)
    if len(fd_list) >= max_idx+1:
        return [*range(max_idx+1, max_idx+1+num_new_idx)]
    else:
        missing_idx = [i for i in range(min_idx, max_idx+1) if i not in idx_list]
        if len(missing_idx) >= num_new_idx:
            print("========== More New Cases Needed To Fill Up All Missing Indices ==========")
            return missing_idx[:num_new_idx]
        else:
            return missing_idx + [*range(max_idx+1, max_idx + 1 + num_new_idx - len(missing_idx))]

In [14]:
def add_unsimulated_log(index, file_path):
    f = open(file_path, 'r+', newline='')
    reader = csv.reader(f)
    lst = list(reader)
    if not [str(index)] in lst:
        writer = csv.writer(f)
        writer.writerow([str(index)])
        print("================ Design Iteration", i, "Added to Log! ================")
    f.close()

In [15]:
# start GH file using Rhino and Grasshopper
fl_path = os.path.abspath('').replace("\\", "/")+"/GH_active_flag.txt"
GH_file_pth = os.path.abspath('').replace("\\", "/")+"/Rhino_File_Generator_10122022.gh"

# record the old flag number
old_fl_file = open(fl_path, 'r')
old_flag_num = old_fl_file.read()
old_fl_file.close()
time.sleep(1)

# start Rhino + Grasshopper
shell_process = subprocess.Popen([GH_file_pth],shell=True)

# wait until Grasshopper fully functioning 
while True:
    flag_f = open(fl_path, 'r')
    new_flag_num = flag_f.read()
    if not new_flag_num == old_flag_num:
        flag_f.close()
        break
    flag_f.close()
time.sleep(3)
print("================== Rhino & Grasshopper File Opened ==================")

================== Rhino & Grasshopper File Opened ==================


In [16]:
# NOTE: Define How Many New Iterations To Generate With "Num_New_Iter" Arguement
Num_New_Iter = 93

# CFD_result_folder = os.path.abspath('').replace("\\", "/").replace('Multizone_space_generator', 'CFD_result_data')
CFD_result_folder = "E:/00_Data_Generator/CFD_result_data/"
new_iter_idx = find_new_idx_list(target_folder = CFD_result_folder, num_new_idx = Num_New_Iter)
# unsimulated_log = os.path.abspath('').replace("\\", "/")+"/unsimulated_log.csv"

for i in new_iter_idx: 
    generate_design(num_rms=5, resolution=0.25, result_folder=CFD_result_folder, num_iteration=i)
    # add_unsimulated_log(i, unsimulated_log)
    print("================ Design Iteration", i, "Created! ================\n")
    time.sleep(1)

================ Design Iteration 5007 Created! ================

================ Design Iteration 5008 Created! ================

================ Design Iteration 5009 Created! ================

================ Design Iteration 5010 Created! ================

================ Design Iteration 5011 Created! ================

================ Design Iteration 5012 Created! ================

================ Design Iteration 5013 Created! ================

================ Design Iteration 5014 Created! ================

================ Design Iteration 5015 Created! ================

================ Design Iteration 5016 Created! ================

================ Design Iteration 5017 Created! ================

================ Design Iteration 5018 Created! ================

================ Design Iteration 5019 Created! ================

================ Design Iteration 5020 Created! ================

================ Design Iteration 5021 Created! ================

==========

In [17]:
shell_process.terminate() # Terminate the subprocess
subprocess.Popen("taskkill /F /IM Rhino.exe",shell=True) # close Rhino & GH

<Popen: returncode: None args: 'taskkill /F /IM Rhino.exe'>

## ===Below are Checking Functions===

In [7]:
# IMPORTANT: Run this Cell to correct the wrong room index in the opening files
Raw_data_folder = "E:/00_Data_Generator/CFD_result_data/"
Revise_data_folder = "E:/00_Data_Generator/Opening_Revised_data/CFD_result_data/"

def find_correct_neighbour(opening, axis, all_rooms):
    true_indice = []
    opx1, opx2, opy1, opy2, opz1, opz2, ind1_, ind2_ = opening
    if axis == "x":
        for j, rm in enumerate(all_rooms):
            rx1, rx2, ry1, ry2, rz1, rz2 = rm
            if opx1 == rx1 or opx1 == rx2:
                if opy1 >= ry1 and opy2 <= ry2:
                    true_indice.append(j)
    else:
        for j, rm in enumerate(all_rooms):
            rx1, rx2, ry1, ry2, rz1, rz2 = rm
            if opy1 == ry1 or opy1 == ry2:
                if opx1 >= rx1 and opx2 <= rx2:
                    true_indice.append(j)
    print("Found correct indice: " + str(int(true_indice[0])) + " and " + str(int(true_indice[1])))
    if not len(true_indice) == 2:
        print("Unexpected condition: detacted " + str(len(true_indice)) + " neighbour rooms for this opening! +++++")
    return true_indice

for k in range(3000):
    rm_path = Raw_data_folder + "design_iteration_" + str(k) + "/design_" + str(k) + "_room.csv"
    op_path = Raw_data_folder + "design_iteration_" + str(k) + "/design_" + str(k) + "_open.csv"
    uvw_folder = Raw_data_folder + "design_iteration_" + str(k) + "/all_open_uvw/"
    
    # read room data
    rm_file = open(rm_path,"r")
    rooms = list(csv.reader(rm_file, delimiter = ','))[:-1]
    rooms = np.array(rooms).astype(float)
    rm_file.close()

    op_file = open(op_path,"r")
    openings = list(csv.reader(op_file, delimiter = ','))
    openings = np.array(openings).astype(float)
    op_file.close()
    
    new_openings = copy.deepcopy(openings)
    old_inds = []
    new_inds = []
    updated = False
    for i, op in enumerate(openings):
        x1, x2, y1, y2, z1, z2, ind1, ind2 = op
        # wrong index not happening in outdoor openings
        if ind1 < 0 or ind2 < 0:
            continue
        
        if x1 == x2:
            rm1, rm2 = rooms[int(ind1)], rooms[int(ind2)]
            if rm1[0] == rm2[1]:
                if x1 == rm1[0]:
                    continue
                else:
                    print("Found wrong index in iteration #" + str(k) + " =================")
                    new_ids = find_correct_neighbour(op, "x", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            elif rm1[1] == rm2[0]:
                if x1 == rm1[1]:
                    continue
                else:
                    print("Found wrong index in iteration #" + str(k) + " =================")
                    new_ids = find_correct_neighbour(op, "x", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            else:
                print("Found wrong index in iteration #" + str(k) + " =================")
                new_ids = find_correct_neighbour(op, "x", rooms)
                new_openings[i][6] = new_ids[0]
                new_openings[i][7] = new_ids[1]
                old_inds.append([ind1, ind2])
                new_inds.append(new_ids)
                updated = True
                
        if y1 == y2:
            rm1, rm2 = rooms[int(ind1)], rooms[int(ind2)]
            if rm1[2] == rm2[3]:
                if y1 == rm1[2]:
                    continue
                else:
                    print("Found wrong index in iteration #" + str(k) + " =================")
                    new_ids = find_correct_neighbour(op, "y", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            elif rm1[3] == rm2[2]:
                if y1 == rm1[3]:
                    continue
                else:
                    print("Found wrong index in iteration #" + str(k) + " =================")
                    new_ids = find_correct_neighbour(op, "y", rooms)
                    new_openings[i][6] = new_ids[0]
                    new_openings[i][7] = new_ids[1]
                    old_inds.append([ind1, ind2])
                    new_inds.append(new_ids)
                    updated = True
            else:
                print("Found wrong index in iteration #" + str(k) + " =================")
                new_ids = find_correct_neighbour(op, "y", rooms)
                new_openings[i][6] = new_ids[0]
                new_openings[i][7] = new_ids[1]
                old_inds.append([ind1, ind2])
                new_inds.append(new_ids)
                updated = True
    if updated:
        param_folder = Revise_data_folder + "design_iteration_" + str(k) + "/"
        if not os.path.exists(param_folder):
            os.makedirs(param_folder)
        revised_opening_file = param_folder + "design_" + str(k) + "_open.csv"
        with open(revised_opening_file,"w",newline='') as csvfile_op: 
            writer_op = csv.writer(csvfile_op)
            for q in range(len(new_openings)):
                writer_op.writerow(new_openings[q])
        
        for l in range(len(old_inds)):
            test_old_name1 = uvw_folder + "window_" + str(int(old_inds[l][0])) + "_" + str(int(old_inds[l][1])) + ".csv"
            test_old_name2 = uvw_folder + "window_" + str(int(old_inds[l][1])) + "_" + str(int(old_inds[l][0])) + ".csv"
            new_name = uvw_folder + "window_" + str(int(new_inds[l][0])) + "_" + str(int(new_inds[l][1])) + ".csv"
            
            if os.path.isfile(test_old_name1):
                os.rename(test_old_name1, new_name)
                print("File name changed to: ", new_name)
            elif os.path.isfile(test_old_name2):
                os.rename(test_old_name2, new_name)
                print("File name changed to: ", new_name)
            elif os.path.isfile(new_name):
                print("File name already changed! =========================================")
            else:
                print("Error: Cannot find target uvw file to rename!! +++++++++++++++++++++")

In [8]:
Raw_data_folder = "E:/00_Data_Generator/CFD_result_data/"
ssnv_ids = []
fewop_ids = []
bad_inds = []
skip_inds = [437, 666, 1122, 1445, 2392, 2816, 2843, 2849]
# bad_inds = [129, 146, 327, 462, 474, 489, 532, 568, 606, 625, 633, 730, 847, 1067, 1075, 1167, 1222, 1265, 1326, 1410, 1424, 1463, 1521, 1577, 1596, 1717, 1760, 1940, 2019, 2053, 2074, 2170, 2210, 2232, 2283, 2361, 2382, 2440, 2562, 2597, 2600, 2645, 2692, 2769, 2799, 2901, 2976]
# index_list = [ele for ele in list(range(3000)) if ele not in skip_inds]
index_list = range(3000)
for k in index_list:
    rm_path = Raw_data_folder + "design_iteration_" + str(k) + "/design_" + str(k) + "_room.csv"
    op_path = Raw_data_folder + "design_iteration_" + str(k) + "/design_" + str(k) + "_open.csv"
    
    # read room data
    rm_file = open(rm_path,"r")
    rooms = list(csv.reader(rm_file, delimiter = ','))[:-1]
    rooms = np.array(rooms).astype(float)
    rm_file.close()

    op_file = open(op_path,"r")
    openings = list(csv.reader(op_file, delimiter = ','))
    openings = np.array(openings).astype(float)
    op_file.close()
    
    connected_rid = openings[:, 6:].flatten().astype(int)
    single_side_exist = False
    too_few_out_opening = False
    for rm_id in range(5):
        if connected_rid.tolist().count(rm_id) < 2:
            single_side_exist = True
    min_out_op = 3 
    too_few_out_opening = sum(np.array(connected_rid) < 0) < min_out_op # has less than 3 outdoor opening

    if single_side_exist:
        print("Iteration #" + str(k) + " : Single-sided Room Exists ======")
        ssnv_ids.append(k)
    if too_few_out_opening:
        print("Iteration #" + str(k) + " : Too few out openings ==========")
        fewop_ids.append(k)
    if single_side_exist or too_few_out_opening:
        bad_inds.append(k)

print("Single-sided Room Exists: ")
print(ssnv_ids)
print("Too few out openings: ")
print(fewop_ids)
print("All bad indice: ")
print(bad_inds)

Single-sided Room Exists: 
[]
Too few out openings: 
[]
All bad indice: 
[]


In [7]:
# result_folder = os.path.abspath('').replace("\\", "/").replace('Multizone_space_generator', 'CFD_result_data')
# for i in range(3000):
#     param_folder = result_folder + "/design_iteration_" + str(i) + "/"
#     if not os.path.exists(param_folder):
#         os.makedirs(param_folder)